In [1]:
import sagemaker
from sagemaker import image_uris
import boto3
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!pip install huggingface-hub -Uqq 

In [14]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./OpenMath-Mistral-7B-v0.1-hf")

In [15]:
local_model_path.mkdir(exist_ok=True)
model_name = "nvidia/OpenMath-Mistral-7B-v0.1-hf"
# commit_hash = "0496919ed797ead809f01c373f1a54251b666690"
snapshot_download(repo_id=model_name,  cache_dir=local_model_path)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/5.54k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

'OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57'

In [16]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket() 

In [17]:
s3_model_prefix = "LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf
model_snapshot_path: OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57


In [18]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57/generation_config.json to s3://sagemaker-us-east-1-434444145045/LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf/generation_config.json
upload: OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57/.gitattributes to s3://sagemaker-us-east-1-434444145045/LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf/.gitattributes
upload: OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57/config.json to s3://sagemaker-us-east-1-434444145045/LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf/config.json
upload: OpenMath-Mistral-7B-v0.1-hf/models--nvidia--OpenMath-Mistral-7B-v0.1-hf/snapshots/e378a80b22387a5a30ccbb9feaf3e9b0bc3cfc57/README.md to s3://sagemaker-us-east-1-434444145045/LLM-RAG/workshop/OpenMath-Mistral-7B-v0.1-hf/README.md
upload: Open